# Capstone Project - Housing Amenities and Prices in Seattle Neighborhoods. 

## Introduction  
In the past few years, as the increasing number of workers in high-tech industries in Seattle, the housing market faces sharply rising demand.  There are sixty neighborhoods in Seattle, separated by their zip codes. These neighborhoods vary a lot in both housing prices and types of venues.  When new workers move into Seattle, they usually consider both the affordability of the house and amenities around the house. I create a map to illustrate both categories of information that could help the city dwellers pick the neighborhood that suits their preferences. 


## Data  

I use the following data in my analysis:  
1. the zip code list of Seattle: https://www.zip-codes.com/city/wa-seattle.asp  
2. coordinates of Seattle neighborhoods: Opencage Geocoder API
3. numbers and types of venues in each neighborhood: Foursquare API
4. boundaries of zip code area: ArcGIS Hub
5. average housing prices: www.zillow.com


Note:
* In order to avoid download Seattle venue data and neighborhood coordinate data multiple times, I collect them through codes separated from the main notebook: 
    

#### Python pacakges

In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
!conda install -c conda-forge geocoder --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge xlrd --yes
from pandas import ExcelWriter
from pandas import ExcelFile
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages alre

#### Prepare the Seattle venues dataset, what are the amenities in each neighborhood? 

Venue information is mainly stored in the **seattle_venue.csv** dataset. But the opencage API gives wrong coordinates for five zip codes. I have googled coordinates of  those neighborhoods and saved venue information of them in the **seattle_venue_sup.csv** dataset.

In [5]:
seattle_venues=pd.read_csv('seattle_venues.csv')
seattle_venues['zipcode']=seattle_venues['zipcode'].astype('str')
seattle_venues.drop(['Unnamed: 0'], axis =1,inplace=True)
seattle_venues.head()

,zipcode,lat,long,Venue,Venue Latitude,Venue Longitude,Venue Category
0,98101,47.61076,-122.336181,Monorail Espresso,47.610828,-122.335048,Coffee Shop
1,98101,47.61076,-122.336181,Din Tai Fung Dumpling House,47.612671,-122.335073,Dumpling Restaurant
2,98101,47.61076,-122.336181,The 5th Avenue Theatre,47.608996,-122.334162,Theater
3,98101,47.61076,-122.336181,Victrola Coffee Roasters,47.610895,-122.338952,Coffee Shop
4,98101,47.61076,-122.336181,Veggie Grill,47.610058,-122.336497,Vegetarian / Vegan Restaurant


In [6]:
seattle_venues.shape

(4823, 7)

In [7]:
correct_coordinates=pd.read_csv('corrected_cor.csv')
correct_coordinates['zipcode']=correct_coordinates['zipcode'].astype('str')
need_to_modify_zip=correct_coordinates['zipcode'].values.tolist()


In [8]:
seattle_venues=seattle_venues[~seattle_venues.zipcode.isin(need_to_modify_zip)]
seattle_venues['zipcode'].nunique()#count number of zipcodes

55

In [9]:
seattle_venues_sup=pd.read_csv('seattle_venues_sup.csv')
seattle_venues_sup['zipcode']=seattle_venues_sup['zipcode'].astype('str')
seattle_venues_sup.drop(['Unnamed: 0'], axis =1,inplace=True)
seattle_venues_sup.head()

,zipcode,lat,long,Venue,Venue Latitude,Venue Longitude,Venue Category
0,98111,47.6099,-122.34,Ellenos Real Greek Yogurt,47.608848,-122.340476,Frozen Yogurt Shop
1,98111,47.6099,-122.34,Pike Place Market,47.609467,-122.341465,Market
2,98111,47.6099,-122.34,Pike Place Fish Market,47.608813,-122.340371,Fish Market
3,98111,47.6099,-122.34,Beecher's Handmade Cheese,47.609570,-122.341851,Cheese Shop
4,98111,47.6099,-122.34,World Spice Merchants,47.608653,-122.341405,Herbs & Spices Store


Now let's merge the two datasets together.

In [10]:
seattle_venues=seattle_venues.append(seattle_venues_sup)
seattle_venues['zipcode'].nunique()

60

In [11]:
seattle_venues.head()

,zipcode,lat,long,Venue,Venue Latitude,Venue Longitude,Venue Category
0,98101,47.61076,-122.336181,Monorail Espresso,47.610828,-122.335048,Coffee Shop
1,98101,47.61076,-122.336181,Din Tai Fung Dumpling House,47.612671,-122.335073,Dumpling Restaurant
2,98101,47.61076,-122.336181,The 5th Avenue Theatre,47.608996,-122.334162,Theater
3,98101,47.61076,-122.336181,Victrola Coffee Roasters,47.610895,-122.338952,Coffee Shop
4,98101,47.61076,-122.336181,Veggie Grill,47.610058,-122.336497,Vegetarian / Vegan Restaurant


In [12]:
sale_venues=seattle_venues

In [13]:
sale_venues.head()

,zipcode,lat,long,Venue,Venue Latitude,Venue Longitude,Venue Category
0,98101,47.61076,-122.336181,Monorail Espresso,47.610828,-122.335048,Coffee Shop
1,98101,47.61076,-122.336181,Din Tai Fung Dumpling House,47.612671,-122.335073,Dumpling Restaurant
2,98101,47.61076,-122.336181,The 5th Avenue Theatre,47.608996,-122.334162,Theater
3,98101,47.61076,-122.336181,Victrola Coffee Roasters,47.610895,-122.338952,Coffee Shop
4,98101,47.61076,-122.336181,Veggie Grill,47.610058,-122.336497,Vegetarian / Vegan Restaurant


How many venue categories?

In [14]:
sale_venues['Venue Category'].nunique()

322

In [15]:
sale_venues['Venue Category'].value_counts()

Coffee Shop                                 382
Pizza Place                                 149
Hotel                                       141
Sandwich Place                              133
Bakery                                      123
Park                                        122
Bar                                         104
Cocktail Bar                                103
Ice Cream Shop                               89
Brewery                                      88
Mexican Restaurant                           83
Breakfast Spot                               78
Italian Restaurant                           76
Burger Joint                                 74
Vietnamese Restaurant                        70
Café                                         70
Grocery Store                                69
American Restaurant                          65
Sushi Restaurant                             61
Thai Restaurant                              60
Seafood Restaurant                      

## Cluster the neighborhoods

In [16]:
# one hot encoding
sale_onehot = pd.get_dummies(sale_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood and coordinate columns back to dataframe
sale_onehot['zipcode'] = sale_venues['zipcode'] 
sale_onehot['Latitude']=sale_venues['lat']
sale_onehot['Longitude']=sale_venues['long']
# move the three columns to the first column
cols_to_move = ['zipcode', 'Latitude', 'Longitude']
sale_onehot= sale_onehot[ cols_to_move + [ col for col in sale_onehot.columns if col not in cols_to_move ] ]
#fixed_columns = [sale_onehot.columns[-1]] + list(sale_onehot.columns[:-1])
#sale_onehot = sale_onehot[fixed_columns]

sale_onehot.head()

,zipcode,Latitude,Longitude,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Science Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,History Museum,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lawyer,Library,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Night Market,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Police Station,Polish Restaurant,Pool,Post Office,Poutine Place,Pub,Public Art,Radio Station,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roller Rink,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationer

In [17]:
sale_onehot.shape

(5308, 325)

In [18]:
neighborhood_venues = sale_onehot.groupby('zipcode').mean().reset_index()
venue_grouped=neighborhood_venues.drop(['Latitude','Longitude'],axis=1)
venue_grouped.head()

,zipcode,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Science Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,History Museum,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lawyer,Library,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Night Market,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Police Station,Polish Restaurant,Pool,Post Office,Poutine Place,Pub,Public Art,Radio Station,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roller Rink,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,

In [19]:
num_top_venues = 5

for hood in venue_grouped['zipcode']:
    print("----"+hood+"----")
    temp = venue_grouped[venue_grouped['zipcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----98101----
            venue  freq
0           Hotel  0.10
1     Coffee Shop  0.07
2          Bakery  0.04
3  Breakfast Spot  0.03
4             Spa  0.02


----98102----
            venue  freq
0     Coffee Shop  0.07
1  Ice Cream Shop  0.04
2            Café  0.04
3    Cocktail Bar  0.04
4     Pizza Place  0.03


----98103----
            venue  freq
0     Coffee Shop  0.07
1     Zoo Exhibit  0.06
2             Bar  0.05
3     Pizza Place  0.05
4  Ice Cream Shop  0.05


----98104----
                   venue  freq
0            Coffee Shop  0.08
1           Cocktail Bar  0.06
2  Vietnamese Restaurant  0.06
3                  Hotel  0.05
4     Seafood Restaurant  0.04


----98105----
             venue  freq
0      Coffee Shop  0.06
1   Ice Cream Shop  0.04
2       Restaurant  0.04
3  Thai Restaurant  0.03
4              Pub  0.03


----98106----
           venue  freq
0    Coffee Shop  0.09
1    Pizza Place  0.06
2  Grocery Store  0.05
3   Burger Joint  0.03
4       Pharmacy  0.03


In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['zipcode','Latitude','Longitude']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted[['zipcode','Latitude','Longitude']] = neighborhood_venues[['zipcode','Latitude','Longitude']]

for ind in np.arange(venue_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 3:] = return_most_common_venues(venue_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,zipcode,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,98101,47.610760,-122.336181,Hotel,Coffee Shop,Bakery,Breakfast Spot,Sushi Restaurant,Dumpling Restaurant,Theater,Market,French Restaurant,Rock Club
1,98102,47.621611,-122.321227,Coffee Shop,Cocktail Bar,Ice Cream Shop,Café,Bar,Yoga Studio,Pizza Place,Mexican Restaurant,Sandwich Place,Italian Restaurant
2,98103,47.673749,-122.343934,Coffee Shop,Zoo Exhibit,Bar,Ice Cream Shop,Pizza Place,Pub,Park,Burger Joint,Café,Japanese Restaurant
3,98104,47.600708,-122.331334,Coffee Shop,Cocktail Bar,Vietnamese Restaurant,Hotel,Seafood Restaurant,Breakfast Spot,Sushi Restaurant,Italian Restaurant,Bookstore,Concert Hall
4,98105,47.662934,-122.320552,Coffee Shop,Ice Cream Shop,Restaurant,Grocery Store,Bar,Bubble Tea Shop,Pub,Thai Restaurant,Pet Store,Korean Restaurant
5,98106,47.516871,-122.354830,Coffee Shop,Pizza Place,Grocery Store,Burger Joint,Bank,Fried Chicken Joint,Pharmacy,Convenience Store,Playground,Fast Food Restaurant
6,98107,47.664346,-122.381360,Brewery,Bar,Mexican Restaurant,Coffee Shop,Cocktail Bar,Ice Cream Shop,New American Restaurant,Clothing Store,Sushi Restaurant,Sandwich Place
7,98108,47.567587,-122.322364,Brewery,Coffee Shop,Pizza Place,Food Truck,BBQ Joint,Bar,Pub,Taco Place,Grocery Store,Café
8,98109,47.633123,-122.348679,Coffee Shop,Museum,Mexican Restaurant,Restaurant,Camera Store,Italian Restaurant,Gym / Fitness Center,Park,Pizza Place,Bakery
9,98110,47.636977,-122.524881,Coffee Shop,Pizza Place,Bakery,Ice Cream Shop,Park,Pharmacy,Trail,American Restaurant,Diner,Wine Bar


In [28]:
neighborhoods_venues_sorted.shape

(60, 13)

In [29]:
# set number of clusters
kclusters = 4

venue_grouped_clustering = venue_grouped.drop('zipcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venue_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 0, 2, 2, 2, 1, 2, 2], dtype=int32)

In [30]:
values, counts = np.unique(kmeans.labels_, return_counts=True)
print(values,counts)

[0 1 2 3] [17  9 33  1]


In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [32]:
neighborhoods_venues_sorted.head()

,Cluster Labels,zipcode,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,98101,47.610760,-122.336181,Hotel,Coffee Shop,Bakery,Breakfast Spot,Sushi Restaurant,Dumpling Restaurant,Theater,Market,French Restaurant,Rock Club
1,2,98102,47.621611,-122.321227,Coffee Shop,Cocktail Bar,Ice Cream Shop,Café,Bar,Yoga Studio,Pizza Place,Mexican Restaurant,Sandwich Place,Italian Restaurant
2,2,98103,47.673749,-122.343934,Coffee Shop,Zoo Exhibit,Bar,Ice Cream Shop,Pizza Place,Pub,Park,Burger Joint,Café,Japanese Restaurant
3,0,98104,47.600708,-122.331334,Coffee Shop,Cocktail Bar,Vietnamese Restaurant,Hotel,Seafood Restaurant,Breakfast Spot,Sushi Restaurant,Italian Restaurant,Bookstore,Concert Hall
4,2,98105,47.662934,-122.320552,Coffee Shop,Ice Cream Shop,Restaurant,Grocery Store,Bar,Bubble Tea Shop,Pub,Thai Restaurant,Pet Store,Korean Restaurant


In [189]:
neighborhoods_venues_sorted.shape

(60, 14)

## How to interpret the clusters?

I use the characteristics of the centroid of a cluster to illustrate the frequency of venues in each cluster.

In [33]:
centroids=pd.DataFrame(data=kmeans.cluster_centers_)
centroids.columns=venue_grouped.columns[1:]
centroids

,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Science Building,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,High School,History Museum,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Lawyer,Library,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Night Market,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Police Station,Polish Restaurant,Pool,Post Office,Poutine Place,Pub,Public Art,Radio Station,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roller Rink,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage 

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Cluster Labels']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
centroid_venues= pd.DataFrame(columns=columns)
centroid_venues['Cluster Labels'] = centroids.index
for ind in np.arange(kclusters):
    centroid_venues.iloc[ind, 1:] = return_most_common_venues(centroids.iloc[ind, :], num_top_venues)

centroid_venues

,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Coffee Shop,Hotel,Cocktail Bar,Bakery,Breakfast Spot,Italian Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,French Restaurant
1,1,Park,Coffee Shop,Pizza Place,Vietnamese Restaurant,Brewery,Grocery Store,Gas Station,Playground,Pub,Bakery
2,2,Coffee Shop,Pizza Place,Sandwich Place,Bar,Mexican Restaurant,Burger Joint,Park,Bakery,Ice Cream Shop,Pet Store
3,3,Park,Convenience Store,Coffee Shop,Pizza Place,Gym,Motel,Bus Station,Storage Facility,Baseball Field,Organic Grocery


##### There are 17 neighborhoods in cluster 0, 9 neighborhoods in cluster 1, 33 in cluster 2, and 1 in cluster 3.  
Both cluster 0 and cluster 2 are attractive for city dwellers, featuring cafes, restaurants, and bars. Venues in cluster 0 may cater to customers who prefer a more luxurious lifestyle. 
With the park as the most frequent venue category, neighborhoods in cluster 1 provide good living area for city dwellers who enjoy the beauty of nature. Neighborhoods in cluster 3 seem to have fewer amenities for a city lifestyle.

## Lets mark those neighborhoods on the map to see their locations.

First find Seattle's location

In [68]:
import API_KEY #This is my API key for opencage API
from API_KEY import API_KEY

In [4]:
 
address = "Seattle. Washington" # Formats the place name
url = 'https://api.opencagedata.com/geocode/v1/json?q={}&key={}'.format(address, API_KEY) # Gets the proper url to make the API call
obj = json.loads(requests.get(url).text) # Loads the JSON file in the form of a python dictionary
results = obj['results'] # Extracts the results information out of the JSON file
latitude = results[0]['geometry']['lat'] # Extracts the latitude value
longitude = results[0]['geometry']['lng'] # Extracts the longitude value
[latitude,longitude]

[47.6038321, -122.3300624]

Now add neighborhoods on the map.

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_venues_sorted['Latitude'], neighborhoods_venues_sorted['Longitude'], neighborhoods_venues_sorted['zipcode'], neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='black', #rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters

Not surprisingly, neighborhoods in cluster 0 are in downtown Seattle.

### Now analyze the housing price data.

##### The dataset comes from  https://www.zillow.com/seattle-wa/home-values/. There are only 34 neighborhoods available. 

In [37]:
xl = pd.ExcelFile("seattle-wa.xls") # housing price data
xl.sheet_names

['All Homes',
 'Single Fam',
 'Condo',
 'Top Tier',
 'Middle Tier',
 'Bottom Tier',
 'Duplex',
 'Studio',
 'One Bed',
 'Two Bed',
 'Three Bed',
 'Four Bed',
 'Many Bed']

In [38]:
price_data=xl.parse('All Homes')
price_data.head()

,"Seattle, WA - All Homes",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48
0,Statistic,NaN,NaN,Zillow Home Value Index,NaN,NaN,NaN,NaN,NaN,Listings with price cut (%),NaN,NaN,NaN,Median value per sq. ft. ($),NaN,NaN,NaN,Median price cut (%),NaN,NaN,NaN,Median list price ($),NaN,NaN,NaN,Median sale price ($),NaN,NaN,NaN,Median list price / sq. ft. ($),NaN,NaN,NaN,Median rent list price ($),NaN,NaN,NaN,Median rent list price / sq. ft. ($),NaN,NaN,NaN,Homes foreclosed,NaN,NaN,NaN,Zillow Rent Index,NaN,NaN,NaN
1,Region Name,Region Type,Type,Current,Month Over Month,Quarter Over Quarter,Year Over Year,5 Year Annualized,10 Year Annualized,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year
2,Seattle,city,All Homes,741800,0.0119633,0.0159227,-0.0233292,0.0873,0.0581,0.071,-0.0621197,-0.132457,-0.0438624,638,0,0,0,0.0297,-1.5444e-05,-0.00332061,-0.006,698300,-0.00107296,0.000358166,0.00467626,713900,0.0106172,0.0323933,0.0143507,510,-0.00161673,-0.01217,0.00921816,2600,-0.0280374,-0.037037,0.04,2.1951,0.0286201,0.0173067,0.0569106,---,---,---,---,2650,0.00378788,-0.0221402,0.0114504
3,98101,zipcode,All Homes,687300,0.00758779,0.00737808,-0.0517746,0.083,0.0568,0.12,-0.0710112,-0.0895238,-0.0188889,800,0,0,0,---,---,---,---,735000,0.0809618,0.0777126,-0.109091,---,---,---,---,845,0.0295128,0.0287602,0.0420686,---,---,---,---,---,---,---,---,---,---,---,---,2410,-0.00823045,-0.0282258,-0.0474308
4,98102,zipcode,All Homes,725200,0.00868775,0.00729228,-0.0410737,0.0771,0.0525,0.0476,-0.0982143,-0.162907,-0.0624727,778,0,0,0,---,---,---,---,659000,-0.0237037,0.0544844,-0.174185,776800,0.200804,0.0685007,0.175901,633,0.000385881,-0.0160151,-0.0251632,---,---,---,---,2.7192,0.0119054,0.0228799,0.0574495,0,---,---,-0.0194,2630,0.0313725,0,0.0313725


In [39]:
temp=price_data.iloc[1:]
temp.reset_index(drop=True, inplace=True)
temp.head()

,"Seattle, WA - All Homes",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48
0,Region Name,Region Type,Type,Current,Month Over Month,Quarter Over Quarter,Year Over Year,5 Year Annualized,10 Year Annualized,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year
1,Seattle,city,All Homes,741800,0.0119633,0.0159227,-0.0233292,0.0873,0.0581,0.071,-0.0621197,-0.132457,-0.0438624,638,0,0,0,0.0297,-1.5444e-05,-0.00332061,-0.006,698300,-0.00107296,0.000358166,0.00467626,713900,0.0106172,0.0323933,0.0143507,510,-0.00161673,-0.01217,0.00921816,2600,-0.0280374,-0.037037,0.04,2.1951,0.0286201,0.0173067,0.0569106,---,---,---,---,2650,0.00378788,-0.0221402,0.0114504
2,98101,zipcode,All Homes,687300,0.00758779,0.00737808,-0.0517746,0.083,0.0568,0.12,-0.0710112,-0.0895238,-0.0188889,800,0,0,0,---,---,---,---,735000,0.0809618,0.0777126,-0.109091,---,---,---,---,845,0.0295128,0.0287602,0.0420686,---,---,---,---,---,---,---,---,---,---,---,---,2410,-0.00823045,-0.0282258,-0.0474308
3,98102,zipcode,All Homes,725200,0.00868775,0.00729228,-0.0410737,0.0771,0.0525,0.0476,-0.0982143,-0.162907,-0.0624727,778,0,0,0,---,---,---,---,659000,-0.0237037,0.0544844,-0.174185,776800,0.200804,0.0685007,0.175901,633,0.000385881,-0.0160151,-0.0251632,---,---,---,---,2.7192,0.0119054,0.0228799,0.0574495,0,---,---,-0.0194,2630,0.0313725,0,0.0313725
4,98103,zipcode,All Homes,813200,0.0119215,0.0171974,-0.0279829,0.0846,0.0622,0.067,-0.0974053,-0.165688,-0.0399376,576,0,0,0,---,---,---,---,725000,-0.0129344,-3.44828e-05,-0.0189783,785000,0.0174984,0.016576,0.00692663,548,0.0247948,0.0247878,-0.020089,2650,-0.0185185,-0.0535714,0.06,2.1945,0.0291651,0.00126038,0.042246,---,---,---,---,2810,0.00716846,-0.0140351,0.0181159


In [40]:
new_header=temp.iloc[0]
temp=temp.iloc[2:]
temp.columns=new_header
temp.reset_index(drop=True,inplace=True)
temp.head()

,Region Name,Region Type,Type,Current,Month Over Month,Quarter Over Quarter,Year Over Year,5 Year Annualized,10 Year Annualized,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,Current,Month Over Month,Quarter Over Quarter,Year Over Year
0,98101,zipcode,All Homes,687300,0.00758779,0.00737808,-0.0517746,0.083,0.0568,0.12,-0.0710112,-0.0895238,-0.0188889,800,0,0,0,---,---,---,---,735000,0.0809618,0.0777126,-0.109091,---,---,---,---,845,0.0295128,0.0287602,0.0420686,---,---,---,---,---,---,---,---,---,---,---,---,2410,-0.00823045,-0.0282258,-0.0474308
1,98102,zipcode,All Homes,725200,0.00868775,0.00729228,-0.0410737,0.0771,0.0525,0.0476,-0.0982143,-0.162907,-0.0624727,778,0,0,0,---,---,---,---,659000,-0.0237037,0.0544844,-0.174185,776800,0.200804,0.0685007,0.175901,633,0.000385881,-0.0160151,-0.0251632,---,---,---,---,2.7192,0.0119054,0.0228799,0.0574495,0,---,---,-0.0194,2630,0.0313725,0,0.0313725
2,98103,zipcode,All Homes,813200,0.0119215,0.0171974,-0.0279829,0.0846,0.0622,0.067,-0.0974053,-0.165688,-0.0399376,576,0,0,0,---,---,---,---,725000,-0.0129344,-3.44828e-05,-0.0189783,785000,0.0174984,0.016576,0.00692663,548,0.0247948,0.0247878,-0.020089,2650,-0.0185185,-0.0535714,0.06,2.1945,0.0291651,0.00126038,0.042246,---,---,---,---,2810,0.00716846,-0.0140351,0.0181159
3,98104,zipcode,All Homes,606800,0.0120578,0.0129127,-0.025664,0.0838,0.0474,---,---,---,---,654,0,0,0,---,---,---,---,693900,0.00734526,0.00734526,0.0280593,---,---,---,---,811,-0.00704918,0.0476694,-0.0541519,---,---,---,---,---,---,---,---,0,---,---,---,2410,0.00416667,-0.00413223,0.0168776
4,98105,zipcode,All Homes,991000,0.0048518,0.0117219,-0.0236257,0.0805,0.0594,0.0423,-0.101977,-0.125685,-0.135829,570,0,0,0,---,---,---,---,849900,0.039633,-0.102511,0.0301818,1100900,0.0016377,0.297772,0.082604,526,-0.00611888,-0.00904995,-0.0339314,2950,-0.0117253,0.0300279,0.0640216,---,---,---,---,0,---,---,---,3130,0.00320513,-0.0515152,0.00967742


In [41]:
seattle_sale=temp.iloc[:,[0,3]]
seattle_sale.columns=['zipcode','price']
seattle_sale.reset_index(drop=True,inplace=True)
seattle_sale.head()

,zipcode,price
0,98101,687300
1,98102,725200
2,98103,813200
3,98104,606800
4,98105,991000


In [42]:
seattle_sale.shape

(34, 2)

#### Now merge the neighborhoods_venues_sorted data with housing sale data.

In [43]:
seattle_mega=pd.merge(neighborhoods_venues_sorted,seattle_sale, on='zipcode')
seattle_mega['price']=seattle_mega['price'].astype('float')
seattle_mega

,Cluster Labels,zipcode,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,price
0,0,98101,47.610760,-122.336181,Hotel,Coffee Shop,Bakery,Breakfast Spot,Sushi Restaurant,Dumpling Restaurant,Theater,Market,French Restaurant,Rock Club,687300.0
1,2,98102,47.621611,-122.321227,Coffee Shop,Cocktail Bar,Ice Cream Shop,Café,Bar,Yoga Studio,Pizza Place,Mexican Restaurant,Sandwich Place,Italian Restaurant,725200.0
2,2,98103,47.673749,-122.343934,Coffee Shop,Zoo Exhibit,Bar,Ice Cream Shop,Pizza Place,Pub,Park,Burger Joint,Café,Japanese Restaurant,813200.0
3,0,98104,47.600708,-122.331334,Coffee Shop,Cocktail Bar,Vietnamese Restaurant,Hotel,Seafood Restaurant,Breakfast Spot,Sushi Restaurant,Italian Restaurant,Bookstore,Concert Hall,606800.0
4,2,98105,47.662934,-122.320552,Coffee Shop,Ice Cream Shop,Restaurant,Grocery Store,Bar,Bubble Tea Shop,Pub,Thai Restaurant,Pet Store,Korean Restaurant,991000.0
5,2,98106,47.516871,-122.354830,Coffee Shop,Pizza Place,Grocery Store,Burger Joint,Bank,Fried Chicken Joint,Pharmacy,Convenience Store,Playground,Fast Food Restaurant,524900.0
6,2,98107,47.664346,-122.381360,Brewery,Bar,Mexican Restaurant,Coffee Shop,Cocktail Bar,Ice Cream Shop,New American Restaurant,Clothing Store,Sushi Restaurant,Sandwich Place,782600.0
7,1,98108,47.567587,-122.322364,Brewery,Coffee Shop,Pizza Place,Food Truck,BBQ Joint,Bar,Pub,Taco Place,Grocery Store,Café,579500.0
8,2,98109,47.633123,-122.348679,Coffee Shop,Museum,Mexican Restaurant,Restaurant,Camera Store,Italian Restaurant,Gym / Fitness Center,Park,Pizza Place,Bakery,740900.0
9,0,98112,47.626709,-122.306787,Coffee Shop,Italian Restaurant,Cocktail Bar,Bakery,Sushi Restaurant,Park,Garden,Scenic Lookout,American Restaurant,Indian Restaurant,1249200.0


In [44]:
cluster_price=seattle_mega[['Cluster Labels','price']].groupby(['Cluster Labels'],as_index=False).mean()
cluster_price

,Cluster Labels,price
0,0,787416.666667
1,1,688837.500000
2,2,658410.526316
3,3,411600.000000


In [45]:
# download countries geojson file
!wget --quiet https://opendata.arcgis.com/datasets/e6c555c6ae7542b2bdec92485892b6e6_113.geojson

In [60]:
zip_list=seattle_mega['zipcode'].tolist() #neighborhoods_venues_sorted['zipcode'].tolist() 
zip_list

['98101',
 '98102',
 '98103',
 '98104',
 '98105',
 '98106',
 '98107',
 '98108',
 '98109',
 '98112',
 '98115',
 '98116',
 '98117',
 '98118',
 '98119',
 '98121',
 '98122',
 '98125',
 '98126',
 '98133',
 '98134',
 '98136',
 '98144',
 '98146',
 '98148',
 '98155',
 '98164',
 '98166',
 '98168',
 '98177',
 '98178',
 '98188',
 '98198',
 '98199']

In [61]:
with open('e6c555c6ae7542b2bdec92485892b6e6_113.geojson', 'r') as jsonFile:
    geo_data = json.load(jsonFile)

In [62]:
tmp = geo_data
# remove ZIP codes not in geo data
geozips = []
for i in range(len(tmp['features'])):
    if tmp['features'][i]['properties']['ZIPCODE'] in list(zip_list):
        geozips.append(tmp['features'][i])
# creating new JSON object
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips
# save uodated JSON object
open("cleaned_geodata.json", "w").write(json.dumps(new_json, sort_keys=True, indent=4, separators=(',', ': ')))

3618622

In [63]:
with open('cleaned_geodata.json', 'r') as jsonFile:
    seattle_geo = json.load(jsonFile)

In [64]:
type(seattle_geo)

dict

In [65]:
seattle_map = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='Mapbox Bright')

In [66]:
# generate choropleth map.
seattle_map.choropleth(
    geo_data=seattle_geo,
    data=seattle_sale,
    columns=['zipcode', 'price'],
    key_on='feature.properties.ZIPCODE',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='House Price',
    #reset=True
)
seattle_map

In [67]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(seattle_mega['Latitude'], seattle_mega['Longitude'], seattle_mega['zipcode'], seattle_mega['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='darkgreen', #rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(seattle_map)
seattle_map


The graph above shows the housing price and amenity cluster of each neighborhood.  

## Result and Discussion

Seattle provides neighborhoods of various characteristics that could satisfy the needs of different people. For people who enjoy metropolis life and people prefer the beauty of nature, there are expensive houses around central Seattle and more affordable choices in other neighborhoods.  

There are two limitations to our data. First,  there are only 34 neighborhoods with housing price information. Second, an ideal dataset of this analysis should use amenity and housing price information for each housing transaction. My study thus only provides limited guidance for people to search for a new home in the Seattle area. 